<a href="https://colab.research.google.com/github/yeee457984/RNN-Reuters_contentclassifier/blob/main/Textual_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

if not os.path.isdir("HappyML"):
  os.system("git clone https://github.com/cnchi/HappyML.git")

In [ ]:
from tensorflow.keras.datasets import reuters

most_freq_words = 10000
(X_train, Y_train), (X_test, Y_test) = reuters.load_data(num_words=most_freq_words)

print("X_train[0]:", X_train[0])
print("Y_train[0]:", Y_train[0])

X_train[0]: [1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]
Y_train[0]: 3


In [ ]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_news = ' '.join([reverse_word_index.get(i - 3, 'NiD') for i in X_train[0]])

print("Decoded text:", decoded_news)

Decoded text: NiD NiD NiD said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3


Preprocessing

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

max_len = 500

X_train = pad_sequences(
        sequences=X_train,
        maxlen=max_len,
        padding="pre",
        truncating="post",
        value=0)

X_test = pad_sequences(
        sequences=X_test,
        maxlen=max_len,
        padding="pre",
        truncating="post",
        value=0)

Y_train = to_categorical(Y_train, num_classes=46)
Y_test = to_categorical(Y_test, num_classes=46)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()

model.add(layers.Embedding(input_dim=most_freq_words, output_dim=128, input_length=max_len))
model.add(layers.Dropout(0.2))

model.add(layers.GRU(40, activation="relu"))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(46, activation="softmax"))

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 dropout_5 (Dropout)         (None, 500, 128)          0         
                                                                 
 gru_3 (GRU)                 (None, 40)                20400     
                                                                 
 dropout_6 (Dropout)         (None, 40)                0         
                                                                 
 dense_2 (Dense)             (None, 46)                1886      
                                                                 
Total params: 1302286 (4.97 MB)
Trainable params: 1302286 (4.97 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
%load_ext tensorboard

!rm -rf ./logs/

import os
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard

logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

In [ ]:
%tensorboard --logdir logs

In [ ]:
model.fit(X_train, Y_train, validation_split=0.2, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
225/225 [==============================] - 71s 305ms/step - loss: 2.6148 - acc: 0.3296 - val_loss: 2.0537 - val_acc: 0.3834
Epoch 2/20
225/225 [==============================] - 66s 291ms/step - loss: 2.0504 - acc: 0.4454 - val_loss: 1.9066 - val_acc: 0.4830
Epoch 3/20
225/225 [==============================] - 67s 297ms/step - loss: 1.8427 - acc: 0.4960 - val_loss: 1.7494 - val_acc: 0.5409
Epoch 4/20
225/225 [==============================] - 62s 276ms/step - loss: 1.7148 - acc: 0.5402 - val_loss: 1.6685 - val_acc: 0.5737
Epoch 5/20
225/225 [==============================] - 65s 288ms/step - loss: 1.6076 - acc: 0.5865 - val_loss: 1.5719 - val_acc: 0.5960
Epoch 6/20
225/225 [==============================] - 64s 285ms/step - loss: 1.5198 - acc: 0.6074 - val_loss: 1.5216 - val_acc: 0.6055
Epoch 7/20
225/225 [==============================] - 62s 276ms/step - loss: 1.4410 - acc: 0.6262 - val_loss: 1.5550 - val_acc: 0.5921
Epoch 8/20
225/225 [==============================] - 6

In [ ]:
model.fit(X_train, Y_train, validation_split=0.2, epochs=6)

Epoch 1/6
225/225 [==============================] - 66s 282ms/step - loss: 2.6845 - acc: 0.3648 - val_loss: 2.0456 - val_acc: 0.4585
Epoch 2/6
225/225 [==============================] - 60s 269ms/step - loss: 1.9864 - acc: 0.4586 - val_loss: 1.8662 - val_acc: 0.4624
Epoch 3/6
225/225 [==============================] - 62s 276ms/step - loss: 1.8385 - acc: 0.5041 - val_loss: 1.7555 - val_acc: 0.5270
Epoch 4/6
225/225 [==============================] - 60s 269ms/step - loss: 1.7297 - acc: 0.5381 - val_loss: 1.7153 - val_acc: 0.5515
Epoch 5/6
225/225 [==============================] - 60s 269ms/step - loss: 1.6321 - acc: 0.5766 - val_loss: 1.6114 - val_acc: 0.6010
Epoch 6/6
225/225 [==============================] - 64s 286ms/step - loss: 1.5507 - acc: 0.5954 - val_loss: 1.5509 - val_acc: 0.6160


In [ ]:
test_loss, test_acc = model.evaluate(X_test, Y_test)
print("Loss of Test Set:", test_loss)
print("Accuracy of Test Set:", test_acc)

71/71 [==============================] - 3s 45ms/step - loss: 1.6193 - acc: 0.5993
Loss of Test Set: 1.6193408966064453
Accuracy of Test Set: 0.5992876291275024
